<a href="https://colab.research.google.com/github/ljs7463/AnalysisProject/blob/master/%EB%8D%B0%EC%9D%B4%EC%BD%98/analytics/Untitled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import lib & data & func

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

# 시각화 폰트 설정
if os.name =='posix':
    plt.rc("font", family = "AppleGothic")

else:
    plt.rc("font", family = "Malgun Gothic")

# 경고문자 무시
warnings.filterwarnings(action='ignore')

In [2]:
# 쥬피터 실행시
# train_path = '../data/open/train.csv'
# test_path = '../data/open/test.csv'
# info_path = '../data/open/data_info.csv'
# sub_path = '../data/open/sample_submission.csv'
# df_train = pd.read_csv(train_path)
# df_test = pd.read_csv(test_path)
# df_info = pd.read_csv(info_path)
# df_sub = pd.read_csv(sub_path)

# 코랩 실행시
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_sub = pd.read_csv('sample_submission.csv')
df_info = pd.read_csv('data_info.csv')

user_id:	사용자의 고유 식별자

subscription_duration:	사용자가 서비스에 가입한 기간 (월)

recent_login_time:	사용자가 마지막으로 로그인한 시간 (일)

average_login_time: 	사용자의 일반적인 로그인 시간

average_time_per_learning_session:	각 학습 세션에 소요된 평균 시간 (분)

monthly_active_learning_days:	월간 활동적인 학습 일수

total_completed_courses:	완료한 총 코스 수

recent_learning_achievement: 	최근 학습 성취도

abandoned_learning_sessions:	중단된 학습 세션 수

community_engagement_level:	커뮤니티 참여도

preferred_difficulty_level:	선호하는 난이도

subscription_type:	구독 유형

customer_inquiry_history:	고객 문의 이력

payment_pattern
  사용자의 지난 3개월 간의 결제 패턴을 10진수로 표현한 값.
  - 7: 3개월 모두 결제함
  - 6: 첫 2개월은 결제했으나 마지막 달에는 결제하지 않음
  - 5: 첫 달과 마지막 달에 결제함
  - 4: 첫 달에만 결제함
  - 3: 마지막 2개월에 결제함
  - 2: 가운데 달에만 결제함
  - 1: 마지막 달에만 결제함
  - 0: 3개월 동안 결제하지 않음

target:	사용자가 다음 달에도 구독을 계속할지 (1) 또는 취소할지 (0)를 나타냄


In [3]:
#  최근 학습 성취도 범주화
def ach(x):
    if x>=80:
        x = 1
    elif x >=60 and x <80:
        x = 2
    elif x >=40 and x <60:
        x = 3
    elif x >=20 and x <40:
        x = 4
    elif x < 20:
        x = 5
    return x

# Feature Engineering

## 구독 유형&난이도 스케일링

In [4]:
## Label Encoding

## train데이터
df_train['preferred_difficulty_level'] = pd.factorize(df_train['preferred_difficulty_level'])[0]
df_train['subscription_type'] = pd.factorize(df_train['subscription_type'])[0]

## test데이터
df_test['preferred_difficulty_level'] = pd.factorize(df_test['preferred_difficulty_level'])[0]
df_test['subscription_type'] = pd.factorize(df_test['subscription_type'])[0]

In [5]:
## train 데이터
df_train['recent_learning_achievement'] = df_train['recent_learning_achievement'].apply(lambda x: ach(x))

## test 데이터
df_test['recent_learning_achievement'] = df_test['recent_learning_achievement'].apply(lambda x: ach(x))

## 아웃라이어 있는컬럼 모두 스케일링

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for col in ['subscription_duration','recent_login_time','average_login_time','average_time_per_learning_session','monthly_active_learning_days','total_completed_courses','recent_learning_achievement']:
    df_train[col] = scaler.fit_transform(df_train[[col]])



## Label Encoding
df_train['preferred_difficulty_level'] = pd.factorize(df_train['preferred_difficulty_level'])[0]
df_train['subscription_type'] = pd.factorize(df_train['subscription_type'])[0]

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for col in ['subscription_duration','recent_login_time','average_login_time','average_time_per_learning_session','monthly_active_learning_days','total_completed_courses','recent_learning_achievement']:
    df_test[col] = scaler.fit_transform(df_test[[col]])



## Label Encoding
df_test['preferred_difficulty_level'] = pd.factorize(df_test['preferred_difficulty_level'])[0]
df_test['subscription_type'] = pd.factorize(df_test['subscription_type'])[0]

# Before Modeling

In [8]:
# Delete user_id
df_train = df_train.drop(columns = 'user_id')

# split target
x = df_train[list(df_train.columns[:-1])]
y = df_train['target']

In [9]:
# Delete user_id
df_test = df_test.drop(columns = 'user_id')

# split target
new_x = df_test


# Modeling

- average_time_per_learning_session : 카테고리화 혹은 아웃라이어 제거후 진행도 해보기
- 선호하는 난이도와 구독유형만 카테고리화

첫 번째 실험
- 선호하는 난이도와 구독유형만 카테고리화
- catboost & gridsearch cv : 0.50548

## catBoost

### gridSearchCV  포함

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score
import pandas as pd


# CatBoost 모델 설정
model = CatBoostClassifier(verbose=False)

# 탐색할 하이퍼파라미터 그리드 정의
param_grid = {
    'iterations': [100, 200],
    'learning_rate': [0.01, 0.1],
    'depth': [3, 4, 5]
}

# Macro F1 스코어를 사용하기 위한 스코어러 생성
macro_f1_scorer = make_scorer(f1_score, average='macro')

# GridSearchCV 설정
grid_search = GridSearchCV(model, param_grid, cv=StratifiedKFold(5), scoring=macro_f1_scorer, n_jobs=-1)

# GridSearchCV 실행
grid_search.fit(x, y)

# 결과 출력
print("최적의 파라미터:", grid_search.best_params_)
print("최고 평균 Macro F1 점수:", grid_search.best_score_)

최적의 파라미터: {'depth': 5, 'iterations': 200, 'learning_rate': 0.1}
최고 평균 Macro F1 점수: 0.4180639281423656


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score
import pandas as pd


# CatBoost 모델 설정
model = CatBoostClassifier(verbose=False)

param_grid = {
    'iterations': [700,800,900,1000],
    'learning_rate': [ 0.05, 0.1, 0.15],
    'depth': [8,9,10,11],

}

# Macro F1 스코어를 사용하기 위한 스코어러 생성
macro_f1_scorer = make_scorer(f1_score, average='macro')

# GridSearchCV 설정
grid_search = GridSearchCV(model, param_grid, cv=StratifiedKFold(5), scoring=macro_f1_scorer, n_jobs=-1)

# GridSearchCV 실행
grid_search.fit(x, y)

# 결과 출력
print("최적의 파라미터:", grid_search.best_params_)
print("최고 평균 Macro F1 점수:", grid_search.best_score_)

KeyboardInterrupt: 

In [ ]:
# Label Encoding
df_test['preferred_difficulty_level'] = pd.factorize(df_test['preferred_difficulty_level'])[0]
df_test['subscription_type'] = pd.factorize(df_test['subscription_type'])[0]
df_test['recent_learning_achievement'] = df_test['recent_learning_achievement'].apply(lambda x: ach(x))

df_test = df_test.drop(columns = 'user_id')
# 최적의 모델 가져오기
best_model = grid_search.best_estimator_
new_x = df_test[list(df_test)]
# 새로운 데이터에 대한 예측 수행
# 최적의 모델 가져오기
best_model = grid_search.best_estimator_
predictions = best_model.predict(new_x)


# 예측값 저장
df_sub['target'] = predictions
df_sub.set_index('user_id').to_csv('csv.csv', encoding="cp949")

## xgBoost

In [10]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, f1_score
from sklearn.datasets import load_iris
import pandas as pd

# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

param_grid = {
    'max_depth': [5,7 ,9, 11],
    'learning_rate': [0.1, 0.01, 0.05],
    'n_estimators': [ 200, 300, 400, 500],
    'colsample_bytree': [0.3, 0.7, 0.9]
}

# XGBoost 분류기 초기화
xgb_classifier = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Macro F1 스코어를 사용하기 위한 스코어러 생성
macro_f1_scorer = make_scorer(f1_score, average='macro')

# GridSearchCV 설정
grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, cv=3, scoring = macro_f1_scorer, n_jobs=-1, verbose=1)

# GridSearchCV 실행
grid_search.fit(x_train, y_train)

# 결과 출력
print("최적의 파라미터:", grid_search.best_params_)
print("최고 평균 정확도:", grid_search.best_score_)

Fitting 3 folds for each of 144 candidates, totalling 432 fits
최적의 파라미터: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 500}
최고 평균 정확도: 0.5003549568555615


In [ ]:
# 최적의 모델 가져오기
best_model = grid_search.best_estimator_

# 예측
y_pred = best_model.predict(x_test)
# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Set Accuracy: {accuracy}")

Test Set Accuracy: 0.549


In [ ]:
# Label Encoding
df_test['preferred_difficulty_level'] = pd.factorize(df_test['preferred_difficulty_level'])[0]
df_test['subscription_type'] = pd.factorize(df_test['subscription_type'])[0]
df_test['recent_learning_achievement'] = df_test['recent_learning_achievement'].apply(lambda x: ach(x))

df_test = df_test.drop(columns = 'user_id')
# 최적의 모델 가져오기
best_model = grid_search.best_estimator_
new_x = df_test[list(df_test)]
# 최적의 모델 가져오기
best_model = grid_search.best_estimator_
prediction = best_model.predict(new_x)


In [ ]:
# 예측값 저장
df_sub['target'] = prediction
df_sub.set_index('user_id').to_csv('csv.csv', encoding="cp949")

# Light GBM

In [11]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, f1_score
from sklearn.datasets import load_iris
import pandas as pd


In [12]:
# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
param_grid = {
    'max_depth': [5,7 ,9, 11],
    'learning_rate': [0.1, 0.01, 0.05],
    'n_estimators': [ 200, 300, 400, 500],
    'colsample_bytree': [0.3, 0.7, 0.9],
    'num_leaves': [31, 50, 70],
}
# LightGBM 분류기 초기화
lgb_classifier = lgb.LGBMClassifier()

# Macro F1 스코어를 사용하기 위한 스코어러 생성
macro_f1_scorer = make_scorer(f1_score, average='macro')

# GridSearchCV 설정s
grid_search = GridSearchCV(estimator=lgb_classifier, param_grid=param_grid, cv=3, scoring=macro_f1_scorer, n_jobs=-1, verbose=1)

# GridSearchCV 실행
grid_search.fit(x_train, y_train)

# 결과 출력
print("최적의 파라미터:", grid_search.best_params_)
print("최고 평균 Macro F1 점수:", grid_search.best_score_)

Fitting 3 folds for each of 432 candidates, totalling 1296 fits
[LightGBM] [Info] Number of positive: 4957, number of negative: 3043
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002012 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 663
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.619625 -> initscore=0.487957
[LightGBM] [Info] Start training from score 0.487957
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

In [13]:
# 최적의 모델 가져오기
best_model = grid_search.best_estimator_

# 예측
y_pred = best_model.predict(x_test)

# Macro F1 점수 평가
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Test Set Macro F1 Score: {macro_f1}")

Test Set Macro F1 Score: 0.48986505336489794


# scalling model

# CNN

In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
num_features = 13
num_samples = 10000

model = models.Sequential()
model.add(layers.Reshape((num_features, 1), input_shape=(num_features,)))
model.add(layers.Conv1D(32, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_3 (Reshape)         (None, 13, 1)             0         
                                                                 
 conv1d_6 (Conv1D)           (None, 11, 32)            128       
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 5, 32)             0         
 g1D)                                                            
                                                                 
 conv1d_7 (Conv1D)           (None, 3, 64)             6208      
                                                                 
 global_max_pooling1d_3 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_3 (Dense)             (None, 1)                

In [20]:
from tensorflow.keras import backend as K

def macro_f1_score(y_true, y_pred, threshold=0.5):
    """Macro F1 score 메트릭을 계산하는 함수."""
    y_pred = K.cast(K.greater(y_pred, threshold), K.floatx())
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p + r + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[macro_f1_score])


# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 모델 학습
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
250/250 [==============================] - 4s 10ms/step - loss: 0.6655 - accuracy: 0.6133 - val_loss: 0.6584 - val_accuracy: 0.6205
Epoch 2/10
250/250 [==============================] - 2s 8ms/step - loss: 0.6592 - accuracy: 0.6183 - val_loss: 0.6644 - val_accuracy: 0.5915
Epoch 3/10
250/250 [==============================] - 1s 6ms/step - loss: 0.6567 - accuracy: 0.6177 - val_loss: 0.6529 - val_accuracy: 0.6205
Epoch 4/10
250/250 [==============================] - 1s 4ms/step - loss: 0.6532 - accuracy: 0.6185 - val_loss: 0.6554 - val_accuracy: 0.6210
Epoch 5/10
250/250 [==============================] - 1s 3ms/step - loss: 0.6517 - accuracy: 0.6180 - val_loss: 0.6556 - val_accuracy: 0.6210
Epoch 6/10
250/250 [==============================] - 1s 3ms/step - loss: 0.6490 - accuracy: 0.6174 - val_loss: 0.6541 - val_accuracy: 0.6110
Epoch 7/10
250/250 [==============================] - 1s 4ms/step - loss: 0.6479 - accuracy: 0.6211 - val_loss: 0.6530 - val_accuracy: 0.6175
Epoch

In [21]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

63/63 - 1s - loss: 0.6695 - accuracy: 0.5360 - 805ms/epoch - 13ms/step

Test accuracy: 0.5360000133514404


In [33]:
# 모델을 사용하여 예측 수행
predictions = model.predict(new_x)

313/313 [==============================] - 1s 3ms/step


In [34]:
predictions

array([[0.45686263],
       [0.5640072 ],
       [0.43610668],
       ...,
       [0.51791155],
       [0.68737906],
       [0.46402428]], dtype=float32)

In [37]:
# 임계값을 기준으로 예측 결과를 0과 1로 변환
binary_predictions = (predictions > 0.5).astype(int)


In [39]:
# 예측값 저장
df_sub['target'] = binary_predictions
df_sub.set_index('user_id').to_csv('cnn.csv', encoding="cp949")

결과 0.51로 현재 최대

## CNN모델 튜닝

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers

num_features = 13
num_samples = 10000

model = models.Sequential()
model.add(layers.Reshape((num_features, 1), input_shape=(num_features,)))
model.add(layers.Conv1D(64, 3, activation='relu'))  # 필터 수 증가
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, 3, activation='relu', kernel_regularizer=regularizers.l2(0.001))) # L2 정규화 추가
# model.add(layers.Conv1D(128, 3, activation='relu')) # 더 많은 필터
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dropout(0.5))                     # 드롭아웃 추가
model.add(layers.Dense(1, activation='sigmoid'))

def macro_f1_score(y_true, y_pred, threshold=0.5):
    """Macro F1 score 메트릭을 계산하는 함수."""
    y_pred = K.cast(K.greater(y_pred, threshold), K.floatx())
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p + r + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[macro_f1_score])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 조기 종료 콜백 추가
model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test), callbacks=[early_stopping])

Epoch 1/20
125/125 [==============================] - 3s 12ms/step - loss: 0.7213 - macro_f1_score: 0.7433 - val_loss: 0.6946 - val_macro_f1_score: 0.7625
Epoch 2/20
125/125 [==============================] - 1s 6ms/step - loss: 0.6951 - macro_f1_score: 0.7586 - val_loss: 0.6822 - val_macro_f1_score: 0.7625
Epoch 3/20
125/125 [==============================] - 1s 6ms/step - loss: 0.6830 - macro_f1_score: 0.7631 - val_loss: 0.6739 - val_macro_f1_score: 0.7625
Epoch 4/20
125/125 [==============================] - 1s 6ms/step - loss: 0.6753 - macro_f1_score: 0.7633 - val_loss: 0.6689 - val_macro_f1_score: 0.7625
Epoch 5/20
125/125 [==============================] - 1s 6ms/step - loss: 0.6724 - macro_f1_score: 0.7628 - val_loss: 0.6638 - val_macro_f1_score: 0.7625
Epoch 6/20
125/125 [==============================] - 1s 6ms/step - loss: 0.6676 - macro_f1_score: 0.7631 - val_loss: 0.6615 - val_macro_f1_score: 0.7625
Epoch 7/20
125/125 [==============================] - 1s 5ms/step - loss: 0

In [11]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

63/63 - 1s - loss: 0.6507 - macro_f1_score: 0.7612 - 580ms/epoch - 9ms/step

Test accuracy: 0.7611902952194214


In [17]:
# 모델을 사용하여 예측 수행
predictions = model.predict(new_x)
# 임계값을 기준으로 예측 결과를 0과 1로 변환
binary_predictions = (predictions > 0.5).astype(int)


313/313 [==============================] - 1s 3ms/step


In [43]:
binary_predictions

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [44]:
# 예측값 저장
df_sub['target'] = binary_predictions
df_sub.set_index('user_id').to_csv('cnn2.csv', encoding="cp949")

# DNN모델

In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from sklearn.metrics import roc_curve, precision_recall_curve, f1_score

num_features = 13
num_samples = 10000

def macro_f1_score(y_true, y_pred, threshold=0.5):
    """Macro F1 score 메트릭을 계산하는 함수."""
    y_pred = K.cast(K.greater(y_pred, threshold), K.floatx())
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p + r + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

# 모델 구축
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(num_features,), kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[macro_f1_score])

# 조기 종료 콜백 설정
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)



# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 조기 종료 콜백 추가
history = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test), callbacks=[early_stopping])

# 모델평가
xtest_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# tmp
y_pred = model.predict(x_test)


# 모델을 사용하여 예측 수행
predictions = model.predict(new_x)

# 정밀도-재현율 곡선 계산
precision, recall, thresholds_pr = precision_recall_curve(y_test, y_pred)

# F1 점수 계산
f1_scores = 2 * (precision * recall) / (precision + recall)
f1_scores = f1_scores[~np.isnan(f1_scores)] # NaN 값 제거
max_f1_index = np.argmax(f1_scores)
best_threshold = thresholds_pr[max_f1_index]

print("최적의 임계값:", best_threshold)

# 최종 예측
final_predictions = (predictions > best_threshold).astype(int)
print("최종 예측 결과:\n", final_predictions)

Epoch 1/20
125/125 [==============================] - 3s 13ms/step - loss: 0.8126 - macro_f1_score: 0.6894 - val_loss: 0.7451 - val_macro_f1_score: 0.7594
Epoch 2/20
125/125 [==============================] - 1s 7ms/step - loss: 0.7436 - macro_f1_score: 0.7370 - val_loss: 0.7337 - val_macro_f1_score: 0.7622
Epoch 3/20
125/125 [==============================] - 1s 10ms/step - loss: 0.7248 - macro_f1_score: 0.7518 - val_loss: 0.7166 - val_macro_f1_score: 0.7625
Epoch 4/20
125/125 [==============================] - 1s 9ms/step - loss: 0.7126 - macro_f1_score: 0.7570 - val_loss: 0.7023 - val_macro_f1_score: 0.7625
Epoch 5/20
125/125 [==============================] - 1s 11ms/step - loss: 0.7012 - macro_f1_score: 0.7620 - val_loss: 0.6943 - val_macro_f1_score: 0.7625
Epoch 6/20
125/125 [==============================] - 1s 9ms/step - loss: 0.6976 - macro_f1_score: 0.7611 - val_loss: 0.6881 - val_macro_f1_score: 0.7625
Epoch 7/20
125/125 [==============================] - 1s 7ms/step - loss:

AttributeError: ignored

In [24]:
# 예측값 저장
df_sub['target'] = final_predictions
df_sub.set_index('user_id').to_csv('cnn2.csv', encoding="cp949")